# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/5. Spark/spark-scripts/section2/Datasets/'

Mounted at /content/drive


In [ ]:
!pip install pyspark
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('agg').getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=842791cfd3e1b3a884aeaac9282f69a94c7a7305cf1c95a2a90c174c07f5270a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [ ]:
import os
loc = 'uw-madison-courses'

df_list = []
for filename in os.listdir(path + loc):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path + loc + '/'+ filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'rooms', 'instructors', 'schedules', 'sections', 'teachings', 'subjects', 'subject_memberships']


Now check the contents of a few of the dataframses that were read in above.

In [ ]:
courses.toPandas()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1
4,f3541888-584a-3923-9ce7-6341ff3d84a1,Cooperative Education/Co-op in Agricultural & ...,1
...,...,...,...
9301,4a7f3d9d-f2e5-3df2-8dee-2c2d02ed9098,Conservation Planning,972
9302,1220244c-2bbf-3169-8eb0-6a2da36f40f6,"Optimizing Value, Quality and Safety in Health...",973
9303,516f637c-a244-3df0-ba3f-f11ce650543d,Patient Education as a Strategy for Advocacy,974
9304,fbe0838f-9469-3d83-ad85-91ba5aabd1a3,Multidisciplinary Approaches to Colon Cancer P...,975


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [ ]:
course_offerings.show(1)
sections.show(1)
courses.show(1)
rooms.show(1)

+--------------------+--------------------+---------+--------------------+
|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
+--------------------+--------------------+---------+--------------------+
only showing top 1 row

+--------------------+--------------------+------------+------+---------+--------------------+
|                uuid|course_offering_uuid|section_type|number|room_uuid|       schedule_uuid|
+--------------------+--------------------+------------+------+---------+--------------------+
|45adf63c-48c9-365...|344b3ebe-da7e-314...|         FLD|     1|     null|f2d66a4d-0c08-3b4...|
+--------------------+--------------------+------------+------+---------+--------------------+
only showing top 1 row

+--------------------+--------------------+------+
|                uuid|                name|number|


In [ ]:
step1 = sections.join(course_offerings, course_offerings.uuid == sections.course_offering_uuid,"inner") \
                  .select(sections.uuid.alias('section_uuid'),sections.room_uuid,course_offerings.course_uuid \
                          ,course_offerings.term_code ,sections.uuid.alias('section_uuid'))
            
step2 = step1.join(courses.select('uuid','name'), courses.uuid == step1.course_uuid)
step3 = step2.join(rooms , rooms.uuid == step2.room_uuid, 'left')
df = step3.select(rooms.uuid, rooms.facility_code, rooms.room_code, step2.term_code,step3.name)

In [ ]:
df.show()

+--------------------+-------------+---------+---------+--------------------+
|                uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|                null|         null|     null|     1092|Cooperative Educa...|
|                null|         null|     null|     1082|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|                null|         null|     null|     1172|Cooperative Educa...|
|                null|         null|     null|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1114|Cooperative Educa...|
|                null|         null|     null|     1114|Cooperative Educa...|
|                null|         null|     null|     1104|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1104|Cooperat

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [ ]:
df.filter('facility_code == 0469').show()

+--------------------+-------------+---------+---------+--------------------+
|                uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     2441|     1152|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1092|  Fundamentals-Flute|
|6af80b0b-b3e3-370...|         0469|     4411|     1172|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1162|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1132|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1072|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1142|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1112|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1102|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1082|  Fundam

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [ ]:
from pyspark.sql.functions import *

#considering one room for each section
df.groupBy('facility_code','name').agg(count('room_code')).show()



+-------------+--------------------+----------------+
|facility_code|                name|count(room_code)|
+-------------+--------------------+----------------+
|       ONLINE| Introduction to Art|               2|
|         0057|Design: Fundament...|              24|
|         0402|Introduction to C...|               1|
|         0482|Russian Honors Tu...|              11|
|         0482|Introductory Topi...|               3|
|         0015|Second Year Compo...|               3|
|         0408|Technical Present...|              52|
|         0048|Intermediate Labo...|               1|
|         0056|The Crusades: Chr...|               1|
|         0400|Introduction to T...|               5|
|         0469|   English Phonology|               5|
|         0470|Principles of Int...|               4|
|         0140|Human Development...|               2|
|         0057|First Semester Ar...|               1|
|         0450|Fundamentals of A...|              18|
|         0408|China in Worl

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [ ]:
fails = grade_distributions.join(course_offerings, grade_distributions.course_offering_uuid == course_offerings.uuid, 'inner')

fails.show(2)

+--------------------+--------------+-------+--------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+-------+--------+--------+-----------+--------------------+--------------------+---------+--------------------+
|course_offering_uuid|section_number|a_count|ab_count|b_count|bc_count|c_count|d_count|f_count|s_count|u_count|cr_count|n_count|p_count|i_count|nw_count|nr_count|other_count|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------+-------+--------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+-------+--------+--------+-----------+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|             1|    105|       0|      0|       0|      0|      0|      0|      0|      0|       0|      0|      0|      0|       0|       0|          0|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
|f718e6c

In [ ]:
fails = fails.groupBy('name').agg(sum('f_count')).withColumnRenamed('sum(f_count)','fails')
fails.show(2)

+--------------------+-----+
|                name|fails|
+--------------------+-----+
|     Math for Tchng:|    8|
|Where Science Mee...|    2|
+--------------------+-----+
only showing top 2 rows



In [ ]:
fails.filter('fails > 0').orderBy(fails.fails.desc()).limit(5).toPandas()

,name,fails
0,Calculus&Analytic Geometry,925
1,Animal Biology,908
2,Intro Organic Chemistry,724
3,Introduction to Psychology,613
4,Calc--Functns of Variables,575


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [ ]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



### Great job! 